# N-gram Model

### José Pablo Kiesling Lange

In [1]:
import re
from collections import Counter
from collections import defaultdict

import nltk
from nltk import bigrams
from nltk.corpus import cess_esp

In [2]:
nltk.download('cess_esp')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package cess_esp to
[nltk_data]     C:\Users\TheKi\AppData\Roaming\nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TheKi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\TheKi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
corpus_nltk = cess_esp.words()

## Estandarización

Para ver la efectivdad de la estandarización, se mostrará las 10 palabras más frecuentes del corpus antes y después de la estandarización. El objetivo es ver si hay modificación en la cantidad de dichas palabras o si una nueva palabra aparece con frecuencia.

In [4]:
def get_most_common_words(corpus, n=10):
    words = [word for line in corpus for word in line.split()]
    most_common = Counter(words).most_common(n)
    return most_common

In [5]:
most_common_words = get_most_common_words(corpus_nltk, n=10)
most_common_words

[(',', 11420),
 ('de', 10234),
 ('la', 6412),
 ('.', 5866),
 ('que', 5552),
 ('el', 5199),
 ('en', 4340),
 ('y', 4235),
 ('*0*', 3883),
 ('"', 3038)]

Además se mostrará los primeros 25 tokens del corpus antes y después de la estandarización.

In [6]:
corpus_nltk[:25]

['El',
 'grupo',
 'estatal',
 'Electricité_de_France',
 '-Fpa-',
 'EDF',
 '-Fpt-',
 'anunció',
 'hoy',
 ',',
 'jueves',
 ',',
 'la',
 'compra',
 'del',
 '51_por_ciento',
 'de',
 'la',
 'empresa',
 'mexicana',
 'Electricidad_Águila_de_Altamira',
 '-Fpa-',
 'EAA',
 '-Fpt-',
 ',']

Como se puede apreciar, hay palabras que tienen `_` en los tokens y separan palabras. Por lo que se harán las funciones específicas para limpiar los tokens y separar las palabras.

In [7]:
def replace_whitespaces(word):
    return "".join(re.sub('_', ' ', word))

In [8]:
def separate_words(word):
    return word.split()

In [9]:
corpus = []

In [10]:
corpus_without_underscore = [replace_whitespaces(word) for word in corpus_nltk]

In [11]:
for word in corpus_without_underscore:
    if len(word.split()) > 1:
        corpus.extend(separate_words(word))
    else:
        corpus.append(word)

Además, se puede ver que hay tokens que empiezan con caracteres no alfanuméricos o que contienen caracteres especiales. Específicamente, los siguientes

In [12]:
set(word for word in corpus if not word.isalnum())

{'!',
 '"',
 '"Guga"',
 '"Steel"',
 '"añicos',
 '%',
 '&',
 '&amp',
 "'",
 "'Arteaga'",
 "'Catanha'",
 "'Erika'",
 "'Guti'",
 "'hooligans'",
 "'matarte'",
 "'savoir",
 "'síndrome",
 '*',
 '*0*',
 '+',
 ',',
 ',obligado',
 '-',
 '-Fpa-',
 '-Fpt-',
 '-LSB-',
 '-fe-',
 '-fpa-',
 '-fpt-',
 '.',
 '...',
 '.264',
 '/',
 "0'5",
 '0,242',
 '0,35',
 '0,5',
 '0,6',
 '0,7',
 '0,77',
 '0,92',
 '0,9349',
 '0,9355',
 '0,9452',
 '0,95',
 '0,9515',
 '0,9554',
 '0-0',
 '0-1',
 '0-1,',
 '0-2',
 '0-3',
 '0.',
 '0.-Athletic',
 '04/20/00-16/00',
 '04/20/04-05/00',
 '04/20/04-08/00',
 '06.00',
 '07.00',
 '07.15',
 '07.28.27',
 '07.30.42',
 '08.00',
 '09.25',
 '09.30',
 '1,04%',
 '1,08',
 '1,1',
 '1,15',
 '1,18',
 '1,26',
 '1,28',
 '1,3',
 '1,32',
 '1,5',
 '1,7',
 '1,75',
 '1,8',
 '1,9',
 '1,96',
 '1-0',
 '1-0.',
 '1-1',
 '1-2',
 '1-3',
 '1-4',
 '1-6',
 '1.',
 '1.000',
 '1.008.000',
 '1.044',
 '1.115.856',
 '1.181',
 '1.200',
 '1.223.112',
 '1.226',
 '1.298',
 '1.300',
 '1.324.000',
 '1.365',
 '1.420',
 '1.5

Como se puede ver, hay palabras que tienen `'`o `"` en los tokens por lo que *solo* se eliminarán esos caracteres y no el resto del token. En los otros casos, no representan alguna palabra, por lo que se eliminarán completamente.

In [13]:
def clean_word(word):
    if re.search(r'[^a-zA-Z0-9\'"áéíóúÁÉÍÓÚ]', word):
        return ""
    
    return re.sub(r'[^a-zA-Z0-9áéíóúÁÉÍÓÚ]', '', word)

In [14]:
corpus = [clean_word(word) for word in corpus]

In [15]:
corpus = [word for word in corpus if clean_word(word) != '']

Finalmente, se pondrán las palabras en minúsculas solo si no es sigla o acrónimo. Para esto, se hará una función que verifique si la palabra está en mayúsculas y si no es así, la pondrá en minúsculas.

In [16]:
def case_folding(corpus):
    return [word.lower() if not word.isupper() else word for word in corpus]

In [17]:
corpus = case_folding(corpus)

In [18]:
most_common_words = get_most_common_words(corpus, n=10)
most_common_words

[('de', 11828),
 ('la', 7150),
 ('el', 6079),
 ('que', 5943),
 ('en', 4991),
 ('y', 4318),
 ('a', 3493),
 ('los', 3229),
 ('del', 2514),
 ('las', 1956)]

In [19]:
corpus[:25]

['el',
 'grupo',
 'estatal',
 'electricité',
 'de',
 'france',
 'EDF',
 'anunció',
 'hoy',
 'jueves',
 'la',
 'compra',
 'del',
 '51',
 'por',
 'ciento',
 'de',
 'la',
 'empresa',
 'mexicana',
 'electricidad',
 'águila',
 'de',
 'altamira',
 'EAA']

Como se puede apreciar, ya solo hay palabras alfanuméricas y con acentos. Y todas están separadas como se debe

## Model 2-gram based

In [20]:
words = nltk.word_tokenize(' '.join(corpus), language='spanish')

In [21]:
bi_grams = list(bigrams(words))

In [22]:
model = defaultdict(lambda: defaultdict(lambda: 0))

Contar ocurrencias para frecuencia relativa

In [23]:
for w1, w2 in bi_grams:
    model[w1][w2] += 1

In [24]:
for w1_w2 in model:
    total = sum(model[w1_w2].values())
    for w2 in model[w1_w2]:
        model[w1_w2][w2] /= total

In [25]:
model['de']['la']

0.1439803855258708

la probabilidad de que la palabra `la` (segunda más frecuente en el corpus) siga a la palabra `de` (la más frecuente en el corpus) es de 0.14